# Modelos 4: Tentando algo diferente
**Nomes**: Bruna Guedes Pereira, Laura Medeiros Dal Ponte, Mariana Melo Pereira

## Introdução

O aprendizado de máquina consolidou-se como uma das ferramentas mais influentes da ciência contemporânea, permitindo que dados complexos promovam decisões automatizadas. Dentro desse campo, os algoritmos de classificação ocupam papel central, pois possibilitam identificar padrões e organizar informações em categorias de forma precisa e eficiente.

Métodos clássicos como Support Vector Machine (SVM), Naive Bayes e Discriminant Analysis possuem relevância histórica na resolução de problemas de classificação, sendo ainda muito utilizados em diversos experimentos científicos. Explorar o funcionamento interno desses algoritmos, compreendendo suas bases matemáticas e avaliando seu desempenho em conjuntos de dados reais, é um exercício essencial de formação acadêmica.

Dessa forma, o presente trabalho dedica-se a investigar o algoritmo Naive Bayes, implementá-lo de maneira didática e comparar seus resultados a um modelo de referência.

A seguir, uma breve contextualização "divertida" para ambientar e motivar os leitores a entender os fundamentos da análise realizada.

### 🎼 **Ricardo e a Harmonia das Fronteiras: A Balada de Bayes**

Ricardo nasceu em uma linhagem de cavaleiros honrados, treinado desde cedo para empunhar espadas e proteger reinos. Mas seu coração sempre bateu em outro ritmo — o das melodias. Inspirado pelas apresentações da Senhorita Creizuda e da Alfaiate Ligeirinha, ele sonhava em ser um **bardo**, em transformar sentimentos em canções e tocar almas com acordes sinceros.

Mesmo preso às expectativas de sua família, Ricardo nunca abandonou sua paixão. Nas noites calmas, ele se refugia na **Floresta Solitária**, onde canta sobre um amor que nunca desiste, nunca desilude, nunca fere. Suas canções ecoam como promessas à vida que deseja, mas que não pode viver por completo.

Foi em uma dessas noites que Ricardo encontrou um antigo manuscrito escondido sob uma raiz de árvore. Nele, estava descrito um algoritmo ancestral, usado por magos analíticos para **decifrar padrões ocultos com surpreendente simplicidade**: o **Naive Bayes**.

Ricardo aprendeu que o Naive Bayes é como um contador de histórias que confia em probabilidades para prever o rumo dos acontecimentos. Ele parte de uma suposição ousada — que cada característica de um dado é **independente das demais**, como notas que soam separadas, mas que juntas formam uma melodia coerente.

A cada nova observação, o algoritmo calcula a **probabilidade de pertencimento a cada classe**, como se perguntasse: “Qual é a chance de esta canção pertencer ao repertório da esperança, ou ao da saudade?” E mesmo com sua simplicidade, o Naive Bayes surpreende pela eficiência, especialmente quando os dados são vastos e as palavras são muitas — como em cartas, poemas ou canções.

Ricardo treinou um modelo Naive Bayes com dados reais, comparando seu desempenho com o de um classificador mais ingênuo. E descobriu que, como em suas baladas, **a beleza pode estar na simplicidade**, e que mesmo suposições aparentemente ingênuas podem revelar verdades profundas.

Mesmo que nunca possa ser um bardo aos olhos do mundo, Ricardo sabe que, com cada linha de código e cada nota tocada ao luar, ele está escrevendo sua própria balada — uma que fala de coragem, de aprendizado, e de nunca desistir daquilo que faz o coração vibrar.

## 🧑‍💻 Vamos, então, para o Código Mágico?

O Código Mágico, capaz de resolver o desafio proposto, será construído a partir da linguagem Python, em conjunto com as suas bibliotecas `scikit-learn`, `pandas` e `matplotlib`. O primeiro passo será, então, importar essas bibliotecas para o nosso ambiente de programação.

## Resultados e Discussões

Ao aplicar o algoritmo Naive Bayes em um conjunto de dados real, foi possível observar seu desempenho em comparação com um classificador baseline, revelando aspectos importantes sobre sua capacidade de generalização e sensibilidade às características dos dados. A implementação em Python puro permitiu compreender detalhadamente cada etapa do processo de classificação, desde o pré-processamento até a tomada de decisão. Os resultados mostraram que, embora o modelo baseline ofereça uma referência útil, o algoritmo estudado apresentou maior precisão e melhor separação entre as classes, especialmente em cenários com dados bem distribuídos. Essa análise reforça a importância de entender o funcionamento interno dos algoritmos para fazer escolhas mais informadas em projetos de aprendizado de máquina.

## Conclusões

Este notebook teve como objetivo compreender e aplicar de forma didática um algoritmo de classificação supervisionada, destacando tanto sua fundamentação teórica quanto sua implementação prática em Python puro. Ao induzir um modelo com o algoritmo escolhido e compará-lo com um classificador baseline, foi possível avaliar seu desempenho e entender suas vantagens e limitações em cenários reais. A construção manual das funções fortaleceu o aprendizado sobre os mecanismos internos do algoritmo, promovendo autonomia na exploração de novas técnicas de aprendizado de máquina. Assim, o material contribui para o desenvolvimento de uma habilidade essencial: a capacidade de estudar e aplicar algoritmos não abordados diretamente em sala de aula.


## Referências Bibliográficas
1. IBM. Máquina de vetores de suporte. Disponível em: <https://www.ibm.com/br-pt/think/topics/support-vector-machine>.
2. OPENAI. ChatGPT. São Francisco: OpenAI, 2025. Disponível em: <https://chat.openai.com/>. Acesso em: 10 set. 2025.

## Uso de Inteligência Artificial

O modelo Microsoft Copilot foi utilizado especificamente para criar a história inicial no estilo RPG e para revisar os textos de desenvolvimento dos exercícios, ajudando a detalhar o passo a passo seguido para guiar melhor o leitor. As perguntas realizadas à IA foram no estilo: *"Escrevi o seguinte texto: ... Como posso melhorá-lo para guiar o leitor passo a passo, mostrando de maneira clara minha resolução?"*.